<a href="https://colab.research.google.com/github/marianna13/Notebooks/blob/master/Hydrogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.special import sph_harm
import seaborn as sns


# constants
h = 1.054e-34
me = 9.1e-31
e = 1.6e-19

# quantum numbers
n = 4
m = 0
l = 3

orbs = {0:'s', 1:'p', 2:'d',3:'f'}

In [0]:
def s(n,l,ro):
  
    a=np.ones(n*3)
    S = 0
    
    assert n>l, "l shouldn't be greater than n!"
    assert l>=m, "m shouldn't be greater than l!"
    
    for k in range(l+1,n+1):
        a[k+1]=(2*(k/n-1))/(k*(k+1)-l*(l+1))*a[k]
        S+=a[k]*np.power(ro,k)
        
    return S
  
  
def F(n,l,ro):
  
    return np.exp(-ro/n)*s(n,l,ro)

  
  
def psi(n,l,m, theta, phi, ro):
    
    return sph_harm(m,l, theta,phi)*F(n,l,ro)/ro
  
  
def prob(n,l,m,theta,phi,ro):
  
    PSI = psi(n,l,m,theta,phi,ro)
    return np.real(PSI*np.conj(PSI))

theta, phi = np.linspace(0, 2 * np.pi, 50), np.linspace(0, np.pi, 50)
THETA, PHI = np.meshgrid(theta, phi)
r = np.linspace(1,2,50)
ro = r*me*e**2/h**2

elements = []
probability = []

for it in theta:
    for ip in phi:
        for ir in ro:
            elements.append(str((it,ip,ir)))
            probability.append(prob(n,l,m,it,ip,ir))          

# Ensure sum of probability is equal to 1
probability = probability/sum(probability)

coord = np.random.choice(elements, size=10000, replace=True, p=probability)
elem_mat = [i.split(',') for i in coord]
elem_mat = np.matrix(elem_mat)

t = [float(i.item()[1:]) for i in elem_mat[:,0]] 
p = [float(i.item()) for i in elem_mat[:,1]] 
r = [float(i.item()[0:-1]) for i in elem_mat[:,2]]

x = r*np.sin(t)*np.cos(p)
y = r*np.sin(t)*np.sin(p) 
z = r*np.cos(t)
                  

In [0]:
# visualising
import plotly.graph_objects as go

fig = go.Figure( go.Scatter3d(x=x, y=y, z=z,mode='markers', marker={'size':0.8, 'opacity':0.8} ) )
fig.update_layout(
    {"title":str(n)+str(orbs[l])+'m = '+str(m)},      
    autosize=False,
    width=1000,
    height=1000,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    )
)

fig.show()